# LLM-as-a-Judge Tutorial

This notebook demonstrates how to use the LLM-as-a-Judge system with structured prompts, variable substitution, and SFT export capabilities.

## Setup and Imports

In [1]:
import sys
import os
# sys.path.append('../../src')  # Add src to path for imports

from llm_utils import (
    LLMJudgeBase, 
    ChainOfThought, 
    TranslationEvaluatorJudge,
    Signature, 
    InputField, 
    OutputField
)
from pydantic import BaseModel
import json

## Example 1: DSPy-like Signature System

First, let's create a simple factual accuracy judge using the Signature system:

In [ ]:
# Define a signature like DSPy (original syntax - shows type warnings)
class FactJudge(Signature):
    """Judge if the answer is factually correct based on the context."""
    
    # Note: The assignments below will show type warnings, but work correctly
    context: str = InputField(desc="Context for the prediction")  # type: ignore
    question: str = InputField(desc="Question to be answered")  # type: ignore
    answer: str = InputField(desc="Answer for the question")  
    factually_correct: bool = OutputField(desc="Is the answer factually correct based on the context?")  # type: ignore

# Show the generated instruction
print("Generated Instruction:")
print(FactJudge.get_instruction())
print("\n" + "="*50 + "\n")

# Show the input/output models
input_model = FactJudge.get_input_model()
output_model = FactJudge.get_output_model()

if input_model is not str:
    print("Input Schema:")
    print(json.dumps(input_model.model_json_schema(), indent=2))

if output_model is not str:
    print("\nOutput Schema:")
    print(json.dumps(output_model.model_json_schema(), indent=2))

Generated Instruction:
Judge if the answer is factually correct based on the context.

**Input Fields:**
- context (str): Context for the prediction
- question (str): Question to be answered
- answer (str): Answer for the question

**Output Fields:**
- factually_correct (bool): Is the answer factually correct based on the context?



Input Schema:
{
  "properties": {
    "context": {
      "description": "Context for the prediction",
      "title": "Context",
      "type": "string"
    },
    "question": {
      "description": "Question to be answered",
      "title": "Question",
      "type": "string"
    },
    "answer": {
      "description": "Answer for the question",
      "title": "Answer",
      "type": "string"
    }
  },
  "required": [
    "context",
    "question",
    "answer"
  ],
  "title": "FactJudgeInput",
  "type": "object"
}

Output Schema:
{
  "properties": {
    "factually_correct": {
      "description": "Is the answer factually correct based on the context?",
    

## Type-Safe Alternative Syntax

The signature system now supports type-safe syntax using `typing.Annotated` to avoid type checker warnings:

In [ ]:
# Import the new type-safe helper functions
from typing import Annotated
from llm_utils import Input, Output

# Type-safe syntax - no warnings!
class FactJudgeTypeSafe(Signature):
    """Judge if the answer is factually correct based on the context."""
    
    context: Annotated[str, Input("Context for the prediction")]
    question: Annotated[str, Input("Question to be answered")]
    answer: Annotated[str, Input("Answer for the question")]
    factually_correct: Annotated[bool, Output("Is the answer factually correct?")]

print("Type-Safe Signature Instruction:")
print(FactJudgeTypeSafe.get_instruction())
print("\n" + "="*50 + "\n")

# Both approaches generate the same schemas
type_safe_input = FactJudgeTypeSafe.get_input_model()
type_safe_output = FactJudgeTypeSafe.get_output_model()

if type_safe_input is not str:
    print("Type-Safe Input Schema:")
    print(json.dumps(type_safe_input.model_json_schema(), indent=2))

if type_safe_output is not str:
    print("\nType-Safe Output Schema:")
    print(json.dumps(type_safe_output.model_json_schema(), indent=2))

## Example 2: Using ChainOfThought with Mock Client

Note: In a real scenario, you would provide an actual OpenAI client or VLLM endpoint.

In [3]:
# For demonstration purposes, let's see how you would use ChainOfThought
# (This requires an actual LLM client to run)

print("Chain of Thought Usage Pattern:")
print("""
# With actual LLM client:
judge = ChainOfThought(FactJudge, client='http://localhost:8000/v1')

# Execute judgment
result = judge(
    context="The sky is blue during daytime due to light scattering.",
    question="What color is the sky?",
    answer="Blue"
)

print(f"Is factually correct: {result.factually_correct}")
""")

Chain of Thought Usage Pattern:

# With actual LLM client:
judge = ChainOfThought(FactJudge, client='http://localhost:8000/v1')

# Execute judgment
result = judge(
    context="The sky is blue during daytime due to light scattering.",
    question="What color is the sky?",
    answer="Blue"
)

print(f"Is factually correct: {result.factually_correct}")



## Example 3: Custom Judge with Template Variables

In [4]:
# Define a custom output model
class QualityScore(BaseModel):
    score: int  # 1-10 rating
    reasoning: str
    categories: list[str]

# Create a judge with template variables
quality_prompt = """
You are a {judge_type} evaluating {content_type}.

Evaluation Criteria:
- {criteria}

Rate the following on a scale of 1-10 and provide reasoning.
Also identify relevant categories from: {categories}

Content to evaluate:
{content}
""".strip()

# Show the template structure
print("Template Variables Required:")
import re
variables = re.findall(r'\{([^}]+)\}', quality_prompt)
for var in set(variables):
    print(f"  - {var}")

print("\nTemplate Preview:")
print(quality_prompt[:200] + "...")

Template Variables Required:
  - criteria
  - content
  - judge_type
  - categories
  - content_type

Template Preview:
You are a {judge_type} evaluating {content_type}.

Evaluation Criteria:
- {criteria}

Rate the following on a scale of 1-10 and provide reasoning.
Also identify relevant categories from: {categories}
...


## Example 4: Translation Evaluator from Raw Code

This demonstrates the TranslationEvaluatorJudge based on your raw code example:

In [ ]:
# Create translation evaluator
evaluator = TranslationEvaluatorJudge()

print("Translation Evaluator System Prompt:")
print(evaluator.system_prompt_template[:300] + "...")

print("\nOutput Schema:")
from llm_utils.lm.llm_as_a_judge import TranslationOutput
print(json.dumps(TranslationOutput.model_json_schema(), indent=2))

print("\nUsage Pattern:")
print("""
# With actual LLM client:
result = evaluator.evaluate_translation(
    source_prompt="Translate this to French: Hello world",
    ai_translation="Bonjour le monde",
    human_reference="Bonjour tout le monde",
    system_message="NONE",
    glossaries=""
)

print(f"Structure Score: {result.structure_score}")
print(f"Translation Score: {result.translation_score}")
print(f"Term Score: {result.term_score}")
""")

## Example 5: SFT Data Collection and Export

In [ ]:
# Create a mock judge for demonstration
mock_judge = LLMJudgeBase(
    system_prompt_template="Rate the sentiment of this text: {text}. Scale: {scale}",
    output_model=str  # Simple string output for demo
)

# Simulate some SFT data
mock_judge.sft_data = [
    {
        'messages': [
            {
                'role': 'system',
                'content': 'Rate the sentiment of this text: I love sunny days! Scale: 1-10'
            },
            {
                'role': 'user',
                'content': 'Please rate the sentiment'
            },
            {
                'role': 'assistant',
                'content': '9 - Very positive sentiment'
            }
        ],
        'variables': {'text': 'I love sunny days!', 'scale': '1-10'},
        'input_data': 'Please rate the sentiment',
        'output': '9 - Very positive sentiment'
    },
    {
        'messages': [
            {
                'role': 'system', 
                'content': 'Rate the sentiment of this text: This is terrible. Scale: 1-10'
            },
            {
                'role': 'user',
                'content': 'Please rate the sentiment'
            },
            {
                'role': 'assistant',
                'content': '2 - Very negative sentiment'
            }
        ],
        'variables': {'text': 'This is terrible', 'scale': '1-10'},
        'input_data': 'Please rate the sentiment',
        'output': '2 - Very negative sentiment'
    }
]

print(f"Collected {len(mock_judge.sft_data)} training examples")

# Test different export formats
formats = ['messages', 'sharegpt', 'full']

for format_name in formats:
    exported = mock_judge.export_sft_data(format_name)
    print(f"\n=== {format_name.upper()} Format ===")
    print(f"Exported {len(exported)} examples")
    print("Sample structure:", list(exported[0].keys()))
    
    if format_name == 'sharegpt':
        print("ShareGPT sample:")
        print(json.dumps(exported[0], indent=2)[:300] + "...")

## Example 6: Batch Processing Pattern

In [ ]:
# Demonstrate how to use the judge system for batch processing
print("Batch Processing Pattern:")
print("""
# Example: Process multiple translations
evaluator = TranslationEvaluatorJudge(client='your-llm-endpoint')

# Sample data
translations = [
    {
        'source': 'Hello world',
        'ai_translation': 'Bonjour le monde', 
        'human_reference': 'Bonjour tout le monde',
        'system_message': 'NONE',
        'glossaries': ''
    },
    # ... more translations
]

# Process in batch
results = []
for item in translations:
    result = evaluator.evaluate_translation(**item)
    results.append(result)
    
# Export all collected SFT data
evaluator.save_sft_data('translation_judge_training_data.json')

# Analyze results
avg_structure = sum(r.structure_score for r in results) / len(results)
avg_translation = sum(r.translation_score for r in results) / len(results)
avg_term = sum(r.term_score for r in results) / len(results)

print(f"Average Scores:")
print(f"  Structure: {avg_structure:.2f}")
print(f"  Translation: {avg_translation:.2f}")
print(f"  Terms: {avg_term:.2f}")
""")

## Example 7: Creating Custom Judge Classes

In [ ]:
# Example of creating a custom judge class
class CodeQualityJudge(LLMJudgeBase):
    """Judge code quality with multiple criteria."""
    
    def __init__(self, **kwargs):
        system_prompt = """
You are an expert code reviewer evaluating {language} code.

Criteria:
- Readability: How easy is it to understand?
- Performance: Are there obvious performance issues?
- Best Practices: Does it follow {language} best practices?
- Security: Are there security concerns?

Code to review:
```{language}
{code}
```

Additional context: {context}
""".strip()
        
        # Define output model
        class CodeReview(BaseModel):
            readability_score: int  # 1-10
            performance_score: int  # 1-10
            best_practices_score: int  # 1-10
            security_score: int  # 1-10
            overall_rating: str  # "excellent", "good", "fair", "poor"
            recommendations: list[str]
            
        super().__init__(
            system_prompt_template=system_prompt,
            output_model=CodeReview,
            **kwargs
        )
    
    def review_code(self, code: str, language: str = 'python', context: str = '') -> dict:
        """Review code with structured output."""
        variables = {
            'code': code,
            'language': language,
            'context': context
        }
        
        results = self.judge(f"Please review this {language} code", variables=variables)
        return results[0]['parsed']

# Show the system prompt template
judge = CodeQualityJudge()
print("Code Quality Judge System Prompt Template:")
print(judge.system_prompt_template)

print("\nUsage Example:")
print("""
# With actual LLM client:
code_judge = CodeQualityJudge(client='your-endpoint')

result = code_judge.review_code(
    code="def add(a, b): return a + b",
    language="python",
    context="Simple utility function"
)

print(f"Overall Rating: {result.overall_rating}")
print(f"Readability: {result.readability_score}/10")
""")

## Summary

This notebook demonstrated the key features of the LLM-as-a-Judge system:

1. **Signature System**: DSPy-like declarative interface for defining input/output schemas
2. **Template Variables**: System prompts with variable substitution
3. **SFT Export**: Automatic collection and export of training data
4. **Chain of Thought**: Built-in reasoning support
5. **Custom Judges**: Easy creation of domain-specific evaluation classes
6. **Multiple Formats**: Support for various export formats (messages, ShareGPT, etc.)

### Next Steps:

1. Set up your LLM endpoint (OpenAI API or VLLM server)
2. Create your own Signature classes for your specific use cases
3. Collect evaluation data and export for fine-tuning smaller models
4. Experiment with different prompt templates and evaluation criteria

### Key Classes:

- `Signature`: Define structured input/output schemas
- `LLMJudgeBase`: Core judge class with template support
- `ChainOfThought`: DSPy-like reasoning wrapper
- `TranslationEvaluatorJudge`: Ready-to-use translation evaluator

The system is designed to be flexible and extensible for various evaluation tasks!